In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from model import SimpleCNN
import os

In [2]:
PATH = os.getcwd()
BS = 50
num_classes = 7


In [3]:
PATH

'/media/evgeniy/Seagate Expansion Drive/код/EmotionRecognition'

In [32]:
transform = {
    'train': transforms.Compose([
        transforms.Resize(48),
        transforms.Grayscale(),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ])
}

In [33]:
data = {
    'train': datasets.ImageFolder('data/train/', transform=transform['train']),
    'test': datasets.ImageFolder('data/test/', transform=transform['train'])
}

In [34]:
train_data_size = len(data['train'])
test_data_size = len(data['test'])
train_data_size, test_data_size

(35640, 247)

In [35]:
train_data = DataLoader(data['train'], shuffle=True, batch_size=256,
                        num_workers=6)
test_data = DataLoader(data['test'], batch_size=247, shuffle=True,
                       num_workers=6)

In [8]:
device = torch.device('cuda:0')

In [16]:
# model = SimpleCNN().to(device)
model = torch.load('EmotionClassificationCNN.4500')

In [17]:
model.class_to_idx

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surpise': 6}

In [18]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=5e-4, nesterov=True, momentum=0.1)
# optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [19]:
data['train'].class_to_idx

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surpise': 6}

In [20]:
model.class_to_idx = data['train'].class_to_idx

In [21]:
model

SimpleCNN(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
  )
  (pred): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=7, bias=True)
    (4): Softmax(dim=None)
  )
)

In [22]:
%%time
for epoch in range(2000): #4100
    print("Epoch: {}/{}".format(epoch+1, 1000))

    model.train()

    for i, (inputs, labels) in enumerate(train_data):

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_func(outputs, labels)

        loss.backward()

        optimizer.step()

    print("Training: Loss: {:.4f}".format(loss.item()))

Epoch: 1/1000
Training: Loss: 1.4464
Epoch: 2/1000
Training: Loss: 1.3788
Epoch: 3/1000
Training: Loss: 1.5112
Epoch: 4/1000
Training: Loss: 1.4119
Epoch: 5/1000
Training: Loss: 1.3164
Epoch: 6/1000
Training: Loss: 1.3993
Epoch: 7/1000
Training: Loss: 1.3731
Epoch: 8/1000
Training: Loss: 1.5004
Epoch: 9/1000
Training: Loss: 1.4642
Epoch: 10/1000
Training: Loss: 1.4695
Epoch: 11/1000
Training: Loss: 1.4252
Epoch: 12/1000
Training: Loss: 1.2764
Epoch: 13/1000
Training: Loss: 1.4095
Epoch: 14/1000
Training: Loss: 1.3192
Epoch: 15/1000
Training: Loss: 1.3290
Epoch: 16/1000
Training: Loss: 1.4792
Epoch: 17/1000
Training: Loss: 1.3940
Epoch: 18/1000
Training: Loss: 1.3778
Epoch: 19/1000
Training: Loss: 1.4326
Epoch: 20/1000
Training: Loss: 1.3884
Epoch: 21/1000
Training: Loss: 1.5136
Epoch: 22/1000
Training: Loss: 1.3435
Epoch: 23/1000
Training: Loss: 1.4326
Epoch: 24/1000
Training: Loss: 1.3777
Epoch: 25/1000
Training: Loss: 1.3804
Epoch: 26/1000
Training: Loss: 1.4097
Epoch: 27/1000
Traini

Training: Loss: 1.4173
Epoch: 215/1000
Training: Loss: 1.4728
Epoch: 216/1000
Training: Loss: 1.3661
Epoch: 217/1000
Training: Loss: 1.3772
Epoch: 218/1000
Training: Loss: 1.4206
Epoch: 219/1000
Training: Loss: 1.4106
Epoch: 220/1000
Training: Loss: 1.5746
Epoch: 221/1000
Training: Loss: 1.4044
Epoch: 222/1000
Training: Loss: 1.3363
Epoch: 223/1000
Training: Loss: 1.4155
Epoch: 224/1000
Training: Loss: 1.3612
Epoch: 225/1000
Training: Loss: 1.4221
Epoch: 226/1000
Training: Loss: 1.4177
Epoch: 227/1000
Training: Loss: 1.5005
Epoch: 228/1000
Training: Loss: 1.3999
Epoch: 229/1000
Training: Loss: 1.4259
Epoch: 230/1000
Training: Loss: 1.4161
Epoch: 231/1000
Training: Loss: 1.3760
Epoch: 232/1000
Training: Loss: 1.4082
Epoch: 233/1000
Training: Loss: 1.3762
Epoch: 234/1000
Training: Loss: 1.4327
Epoch: 235/1000
Training: Loss: 1.4007
Epoch: 236/1000
Training: Loss: 1.4375
Epoch: 237/1000
Training: Loss: 1.3696
Epoch: 238/1000
Training: Loss: 1.4278
Epoch: 239/1000
Training: Loss: 1.4753
Ep

Training: Loss: 1.3773
Epoch: 426/1000
Training: Loss: 1.4701
Epoch: 427/1000
Training: Loss: 1.3703
Epoch: 428/1000
Training: Loss: 1.4568
Epoch: 429/1000
Training: Loss: 1.3957
Epoch: 430/1000
Training: Loss: 1.4220
Epoch: 431/1000
Training: Loss: 1.4545
Epoch: 432/1000
Training: Loss: 1.4835
Epoch: 433/1000
Training: Loss: 1.5630
Epoch: 434/1000
Training: Loss: 1.5219
Epoch: 435/1000
Training: Loss: 1.3451
Epoch: 436/1000
Training: Loss: 1.4162
Epoch: 437/1000
Training: Loss: 1.3562
Epoch: 438/1000
Training: Loss: 1.3992
Epoch: 439/1000
Training: Loss: 1.4557
Epoch: 440/1000
Training: Loss: 1.4079
Epoch: 441/1000
Training: Loss: 1.4247
Epoch: 442/1000
Training: Loss: 1.5255
Epoch: 443/1000
Training: Loss: 1.4567
Epoch: 444/1000
Training: Loss: 1.4339
Epoch: 445/1000
Training: Loss: 1.3906
Epoch: 446/1000
Training: Loss: 1.3940
Epoch: 447/1000
Training: Loss: 1.4896
Epoch: 448/1000
Training: Loss: 1.4021
Epoch: 449/1000
Training: Loss: 1.3829
Epoch: 450/1000
Training: Loss: 1.4521
Ep

Training: Loss: 1.4157
Epoch: 637/1000
Training: Loss: 1.4079
Epoch: 638/1000
Training: Loss: 1.3708
Epoch: 639/1000
Training: Loss: 1.4001
Epoch: 640/1000
Training: Loss: 1.4706
Epoch: 641/1000
Training: Loss: 1.4091
Epoch: 642/1000
Training: Loss: 1.3906
Epoch: 643/1000
Training: Loss: 1.4931
Epoch: 644/1000
Training: Loss: 1.4062
Epoch: 645/1000
Training: Loss: 1.3855
Epoch: 646/1000
Training: Loss: 1.3862
Epoch: 647/1000
Training: Loss: 1.4323
Epoch: 648/1000
Training: Loss: 1.4933
Epoch: 649/1000
Training: Loss: 1.4741
Epoch: 650/1000
Training: Loss: 1.4920
Epoch: 651/1000
Training: Loss: 1.3616
Epoch: 652/1000
Training: Loss: 1.3293
Epoch: 653/1000
Training: Loss: 1.4876
Epoch: 654/1000
Training: Loss: 1.3534
Epoch: 655/1000
Training: Loss: 1.4745
Epoch: 656/1000
Training: Loss: 1.4023
Epoch: 657/1000
Training: Loss: 1.4314
Epoch: 658/1000
Training: Loss: 1.3820
Epoch: 659/1000
Training: Loss: 1.4571
Epoch: 660/1000
Training: Loss: 1.2534
Epoch: 661/1000
Training: Loss: 1.3694
Ep

Training: Loss: 1.4594
Epoch: 848/1000
Training: Loss: 1.4054
Epoch: 849/1000
Training: Loss: 1.3050
Epoch: 850/1000
Training: Loss: 1.4157
Epoch: 851/1000
Training: Loss: 1.3618
Epoch: 852/1000
Training: Loss: 1.4139
Epoch: 853/1000
Training: Loss: 1.3195
Epoch: 854/1000
Training: Loss: 1.4946
Epoch: 855/1000
Training: Loss: 1.3562
Epoch: 856/1000
Training: Loss: 1.3468
Epoch: 857/1000
Training: Loss: 1.2890
Epoch: 858/1000
Training: Loss: 1.4525
Epoch: 859/1000
Training: Loss: 1.4328
Epoch: 860/1000
Training: Loss: 1.3868
Epoch: 861/1000
Training: Loss: 1.4218
Epoch: 862/1000
Training: Loss: 1.4136
Epoch: 863/1000
Training: Loss: 1.3573
Epoch: 864/1000
Training: Loss: 1.3799
Epoch: 865/1000
Training: Loss: 1.4150
Epoch: 866/1000
Training: Loss: 1.4277
Epoch: 867/1000
Training: Loss: 1.3859
Epoch: 868/1000
Training: Loss: 1.4135
Epoch: 869/1000
Training: Loss: 1.2959
Epoch: 870/1000
Training: Loss: 1.4077
Epoch: 871/1000
Training: Loss: 1.3953
Epoch: 872/1000
Training: Loss: 1.3115
Ep

Training: Loss: 1.4239
Epoch: 1057/1000
Training: Loss: 1.4737
Epoch: 1058/1000
Training: Loss: 1.3979
Epoch: 1059/1000
Training: Loss: 1.3787
Epoch: 1060/1000
Training: Loss: 1.3222
Epoch: 1061/1000
Training: Loss: 1.5077
Epoch: 1062/1000
Training: Loss: 1.4315
Epoch: 1063/1000
Training: Loss: 1.3396
Epoch: 1064/1000
Training: Loss: 1.3665
Epoch: 1065/1000
Training: Loss: 1.3597
Epoch: 1066/1000
Training: Loss: 1.3689
Epoch: 1067/1000
Training: Loss: 1.4403
Epoch: 1068/1000
Training: Loss: 1.4178
Epoch: 1069/1000
Training: Loss: 1.3738
Epoch: 1070/1000
Training: Loss: 1.3533
Epoch: 1071/1000
Training: Loss: 1.3898
Epoch: 1072/1000
Training: Loss: 1.3516
Epoch: 1073/1000
Training: Loss: 1.3400
Epoch: 1074/1000
Training: Loss: 1.3238
Epoch: 1075/1000
Training: Loss: 1.5104
Epoch: 1076/1000
Training: Loss: 1.2677
Epoch: 1077/1000
Training: Loss: 1.4170
Epoch: 1078/1000
Training: Loss: 1.5051
Epoch: 1079/1000
Training: Loss: 1.4203
Epoch: 1080/1000
Training: Loss: 1.4748
Epoch: 1081/1000


Training: Loss: 1.4296
Epoch: 1262/1000
Training: Loss: 1.3838
Epoch: 1263/1000
Training: Loss: 1.4122
Epoch: 1264/1000
Training: Loss: 1.3858
Epoch: 1265/1000
Training: Loss: 1.4274
Epoch: 1266/1000
Training: Loss: 1.4057
Epoch: 1267/1000
Training: Loss: 1.4564
Epoch: 1268/1000
Training: Loss: 1.3638
Epoch: 1269/1000
Training: Loss: 1.3329
Epoch: 1270/1000
Training: Loss: 1.3708
Epoch: 1271/1000
Training: Loss: 1.3095
Epoch: 1272/1000
Training: Loss: 1.4474
Epoch: 1273/1000
Training: Loss: 1.5130
Epoch: 1274/1000
Training: Loss: 1.4632
Epoch: 1275/1000
Training: Loss: 1.3454
Epoch: 1276/1000
Training: Loss: 1.4469
Epoch: 1277/1000
Training: Loss: 1.4732
Epoch: 1278/1000
Training: Loss: 1.3596
Epoch: 1279/1000
Training: Loss: 1.4543
Epoch: 1280/1000
Training: Loss: 1.3940
Epoch: 1281/1000
Training: Loss: 1.3459
Epoch: 1282/1000
Training: Loss: 1.3739
Epoch: 1283/1000
Training: Loss: 1.3728
Epoch: 1284/1000
Training: Loss: 1.3996
Epoch: 1285/1000
Training: Loss: 1.4115
Epoch: 1286/1000


Training: Loss: 1.3840
Epoch: 1467/1000
Training: Loss: 1.3678
Epoch: 1468/1000
Training: Loss: 1.4112
Epoch: 1469/1000
Training: Loss: 1.3996
Epoch: 1470/1000
Training: Loss: 1.3848
Epoch: 1471/1000
Training: Loss: 1.4180
Epoch: 1472/1000
Training: Loss: 1.3758
Epoch: 1473/1000
Training: Loss: 1.4028
Epoch: 1474/1000
Training: Loss: 1.3161
Epoch: 1475/1000
Training: Loss: 1.3738
Epoch: 1476/1000
Training: Loss: 1.4178
Epoch: 1477/1000
Training: Loss: 1.3732
Epoch: 1478/1000
Training: Loss: 1.3607
Epoch: 1479/1000
Training: Loss: 1.4167
Epoch: 1480/1000
Training: Loss: 1.3441
Epoch: 1481/1000
Training: Loss: 1.4008
Epoch: 1482/1000
Training: Loss: 1.4181
Epoch: 1483/1000
Training: Loss: 1.3782
Epoch: 1484/1000
Training: Loss: 1.4022
Epoch: 1485/1000
Training: Loss: 1.4030
Epoch: 1486/1000
Training: Loss: 1.3230
Epoch: 1487/1000
Training: Loss: 1.4386
Epoch: 1488/1000
Training: Loss: 1.4632
Epoch: 1489/1000
Training: Loss: 1.3782
Epoch: 1490/1000
Training: Loss: 1.3681
Epoch: 1491/1000


Training: Loss: 1.3427
Epoch: 1672/1000
Training: Loss: 1.4028
Epoch: 1673/1000
Training: Loss: 1.3767
Epoch: 1674/1000
Training: Loss: 1.3411
Epoch: 1675/1000
Training: Loss: 1.3263
Epoch: 1676/1000
Training: Loss: 1.4002
Epoch: 1677/1000
Training: Loss: 1.3385
Epoch: 1678/1000
Training: Loss: 1.2680
Epoch: 1679/1000
Training: Loss: 1.4041
Epoch: 1680/1000
Training: Loss: 1.3677
Epoch: 1681/1000
Training: Loss: 1.3075
Epoch: 1682/1000
Training: Loss: 1.3792
Epoch: 1683/1000
Training: Loss: 1.3677
Epoch: 1684/1000
Training: Loss: 1.4351
Epoch: 1685/1000
Training: Loss: 1.4213
Epoch: 1686/1000
Training: Loss: 1.3341
Epoch: 1687/1000
Training: Loss: 1.3632
Epoch: 1688/1000
Training: Loss: 1.3753
Epoch: 1689/1000
Training: Loss: 1.3437
Epoch: 1690/1000
Training: Loss: 1.3811
Epoch: 1691/1000
Training: Loss: 1.4575
Epoch: 1692/1000
Training: Loss: 1.3441
Epoch: 1693/1000
Training: Loss: 1.3942
Epoch: 1694/1000
Training: Loss: 1.3272
Epoch: 1695/1000
Training: Loss: 1.3416
Epoch: 1696/1000


Training: Loss: 1.3984
Epoch: 1877/1000
Training: Loss: 1.3708
Epoch: 1878/1000
Training: Loss: 1.3142
Epoch: 1879/1000
Training: Loss: 1.3966
Epoch: 1880/1000
Training: Loss: 1.2764
Epoch: 1881/1000
Training: Loss: 1.4103
Epoch: 1882/1000
Training: Loss: 1.4145
Epoch: 1883/1000
Training: Loss: 1.4668
Epoch: 1884/1000
Training: Loss: 1.3904
Epoch: 1885/1000
Training: Loss: 1.4418
Epoch: 1886/1000
Training: Loss: 1.4123
Epoch: 1887/1000
Training: Loss: 1.3853
Epoch: 1888/1000
Training: Loss: 1.3747
Epoch: 1889/1000
Training: Loss: 1.4091
Epoch: 1890/1000
Training: Loss: 1.3753
Epoch: 1891/1000
Training: Loss: 1.3661
Epoch: 1892/1000
Training: Loss: 1.4160
Epoch: 1893/1000
Training: Loss: 1.3797
Epoch: 1894/1000
Training: Loss: 1.4129
Epoch: 1895/1000
Training: Loss: 1.3959
Epoch: 1896/1000
Training: Loss: 1.3074
Epoch: 1897/1000
Training: Loss: 1.3214
Epoch: 1898/1000
Training: Loss: 1.3811
Epoch: 1899/1000
Training: Loss: 1.3174
Epoch: 1900/1000
Training: Loss: 1.4429
Epoch: 1901/1000


In [36]:
for i, l in test_data:
    images = i
    label = l

In [37]:
for i, l in test_data:
    images = i
    label = l

In [38]:
for i, l in test_data:
    images = i
    label = l

In [39]:
for i, l in test_data:
    images = i
    label = l

In [40]:
images.shape, label.shape
pred = model(images.to(device))
pred.shape

torch.Size([247, 7])

In [41]:
torch.tensor([x.argmax() for x in pred])

tensor([4, 2, 3, 5, 0, 4, 2, 4, 3, 5, 4, 5, 5, 4, 4, 5, 4, 4, 2, 5, 4, 3, 0, 3,
        5, 4, 5, 0, 3, 5, 0, 5, 5, 5, 3, 2, 5, 5, 4, 3, 3, 5, 3, 2, 5, 5, 3, 5,
        4, 4, 5, 2, 5, 4, 5, 4, 5, 5, 5, 5, 3, 4, 4, 2, 4, 5, 4, 5, 5, 3, 5, 5,
        4, 4, 2, 4, 0, 3, 2, 4, 3, 5, 2, 4, 5, 3, 3, 5, 3, 5, 5, 3, 3, 2, 5, 5,
        5, 5, 5, 5, 0, 3, 5, 3, 3, 0, 5, 2, 5, 5, 5, 4, 5, 2, 3, 5, 5, 5, 3, 5,
        5, 3, 5, 4, 5, 3, 4, 5, 2, 5, 5, 4, 3, 3, 3, 0, 5, 3, 5, 6, 4, 5, 5, 5,
        6, 5, 5, 4, 4, 2, 3, 5, 6, 5, 3, 3, 4, 0, 3, 5, 3, 3, 4, 4, 5, 5, 0, 3,
        5, 3, 2, 5, 2, 5, 4, 3, 6, 0, 5, 6, 0, 5, 5, 3, 5, 5, 5, 4, 3, 3, 5, 3,
        2, 5, 6, 5, 5, 3, 5, 5, 5, 4, 4, 0, 3, 4, 5, 5, 5, 5, 4, 5, 3, 3, 0, 5,
        4, 5, 4, 5, 5, 0, 4, 5, 5, 5, 5, 6, 4, 5, 3, 5, 5, 5, 4, 0, 6, 5, 4, 3,
        5, 5, 0, 4, 2, 5, 3])

In [43]:
int(sum(torch.tensor([x.argmax() for x in pred]) == label)) / 247

0.29554655870445345

In [31]:
label

tensor([6, 5, 2, 4, 6, 3, 2, 3, 6, 0, 2, 0, 1, 5, 0, 3, 3, 2, 3, 4, 4, 4, 1, 0,
        5, 5, 6, 4, 0, 1, 1, 5, 2, 1, 6])

In [ ]:
a = model(inputs[0:2])
loss_func(a, labels[0:2])

In [55]:
torch.save(model, 'EmotionClassificationCNN.2100')

In [27]:
resnet34.class_to_idx = data['train'].class_to_idx